In [22]:
import simple_salesforce
from os import environ as env

SF = simple_salesforce.Salesforce(env['SF_USER'], env['SF_PASS'], env['SF_TOKEN'])
print(SF)

In [23]:
lookup = {
    'Address__c': 'address',
    'Demo_Contractor_TEXT_ONLY__c': 'contractor_name',
    'Socrata_Projected_Knocked_By_Date__c': 'socrata_demo_date',
    'DEMO_Planned_Knock_Down_Date__c': 'planned_demo_date',
    'DEMO_Knock_Down_Date__c': 'demo_date',
    'Socrata_Reported_Price__c': 'price',
    'DEMO_Open_Hole_Approved_Date__c': 'demo_open_hole',
    'BSEED_Open_Hole_Approved__c': 'bseed_open_hole',
    'DEMO_Winter_Grade_Approved_Date__c': 'demo_winter_grade',
    'BSEED_Winter_Grade_Approved__c': 'bseed_winter_grade',
    'DEMO_Final_Grade_Approved_Date__c': 'demo_final_grade',
    'BSEED_Final_Grade_Approved__c': 'bseed_final_grade',
    'Status': 'status',
    'CaseNumber': 'case_no',
    'Parcel_ID__c': 'parcel_id',
    'RecordTypeID': 'record_type_id',
    'Non_HHF_Commercial_Demo__c': 'comm_demo',
    'DBA_Com_ENV_Group_NumText__c': 'env_group_no',
    'Council_District__c': 'council_district',
    'Neighborhood__c': 'neighborhood',
    'ACCT_Latitude__c': 'latitude',
    'ACCT_Longitude__c': 'longitude'
}

query = """
Select {} from Case where
    DEMO_Knock_Down_Date__c >= 2014-01-01
        AND
    Non_HHF_Commercial_Demo__c='Yes'
""".format(",".join(lookup.keys()))

res = SF.query_all(query)

In [24]:
import pandas
df = pandas.DataFrame.from_records(res['records'])
df.rename(columns=lookup, inplace=True)
df.drop('attributes', inplace=True, axis=1)

print(df.shape)
df

(379, 22)


,address,contractor_name,socrata_demo_date,planned_demo_date,demo_date,price,demo_open_hole,bseed_open_hole,demo_winter_grade,bseed_winter_grade,...,status,case_no,parcel_id,RecordTypeId,comm_demo,env_group_no,council_district,neighborhood,latitude,longitude
0,5313 E Nevada,Dore & Associates,2017-09-07,2017-06-15,2017-06-15,119700.00,None,None,None,None,...,Demolished,00133399,13007967-76,012j0000000uc9hAAA,Yes,None,3,Krainz Woods,42.426387,-83.049461
1,1855 Ferry Park,Dore & Associates,2017-08-22,2017-05-31,2017-05-31,22750.00,None,None,None,None,...,Demolished,00133402,08007626.,012j0000000uc9hAAA,Yes,None,5,Elijah McCoy,42.362463,-83.089753
2,14777 Dexter,DMC Consultants,2017-04-17,2017-04-10,2017-04-10,36840.00,2017-04-21,2017-04-21,None,None,...,Demolished,00133276,12010811.,012j0000000uc9hAAA,Yes,None,2,Dexter-Fenkell,42.398695,-83.135776
3,8402 Fenkell,Able Demolition,2017-08-31,None,2017-06-05,7676.00,None,2017-06-07,None,None,...,Demolished,00133512,16007205.,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402781,-83.153512
4,3930 Michigan Ave,"Adamo Group, Inc.",2017-09-10,None,2017-06-13,35600.00,None,2017-06-16,None,None,...,Demolished,00133513,14000725.,012j0000000uc9hAAA,Yes,None,6,Chadsey Condon,42.331588,-83.097686
5,8408 Fenkell,Able Demolition,2017-08-31,None,2017-06-05,20000.00,None,2017-06-07,None,None,...,Demolished,00133525,16007204.,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402781,-83.153634
6,8418 Fenkell,Able Demolition,2017-08-31,None,2017-05-29,20200.00,None,2017-06-07,None,None,...,Demolished,00133526,16007203.,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402779,-83.153789
7,4860 Pennsylvania,Dore & Associates,2017-09-10,2017-06-16,2017-06-16,36000.00,None,2017-07-20,None,None,...,Demolished,00133555,19001287.,012j0000000uc9hAAA,Yes,None,4,Gratiot Woods,42.381792,-82.997472
8,1920 Collingwood,"Adamo Group, Inc.",2017-09-26,None,2017-06-29,114450.00,None,2017-07-13,None,None,...,Demolished,00133651,08003313-4,012j0000000uc9hAAA,Yes,None,5,Dexter-Linwood,42.383735,-83.105180
9,5942 Cadillac,Rickman Enterprise Group,None,None,2015-10-01,62600.00,2015-10-06,None,None,None,...,Demolished,00098580,19005159-62,012j0000000zM27AAE,Yes,None,4,Gratiot Woods,42.390861,-83.001863


In [25]:
# create new cols - prioritize bseed inspection date, if that's null use demo inspection date
df['open_hole_date'] = df['bseed_open_hole'].fillna(df['demo_open_hole'])
df['winter_grade_date'] = df['bseed_winter_grade'].fillna(df['demo_winter_grade'])
df['final_grade_date'] = df['bseed_final_grade'].fillna(df['demo_final_grade'])
df[:20]

,address,contractor_name,socrata_demo_date,planned_demo_date,demo_date,price,demo_open_hole,bseed_open_hole,demo_winter_grade,bseed_winter_grade,...,RecordTypeId,comm_demo,env_group_no,council_district,neighborhood,latitude,longitude,open_hole_date,winter_grade_date,final_grade_date
0,5313 E Nevada,Dore & Associates,2017-09-07,2017-06-15,2017-06-15,119700.00,None,None,None,None,...,012j0000000uc9hAAA,Yes,None,3,Krainz Woods,42.426387,-83.049461,None,None,None
1,1855 Ferry Park,Dore & Associates,2017-08-22,2017-05-31,2017-05-31,22750.00,None,None,None,None,...,012j0000000uc9hAAA,Yes,None,5,Elijah McCoy,42.362463,-83.089753,None,None,None
2,14777 Dexter,DMC Consultants,2017-04-17,2017-04-10,2017-04-10,36840.00,2017-04-21,2017-04-21,None,None,...,012j0000000uc9hAAA,Yes,None,2,Dexter-Fenkell,42.398695,-83.135776,2017-04-21,None,2017-05-19
3,8402 Fenkell,Able Demolition,2017-08-31,None,2017-06-05,7676.00,None,2017-06-07,None,None,...,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402781,-83.153512,2017-06-07,None,2017-06-08
4,3930 Michigan Ave,"Adamo Group, Inc.",2017-09-10,None,2017-06-13,35600.00,None,2017-06-16,None,None,...,012j0000000uc9hAAA,Yes,None,6,Chadsey Condon,42.331588,-83.097686,2017-06-16,None,2017-06-22
5,8408 Fenkell,Able Demolition,2017-08-31,None,2017-06-05,20000.00,None,2017-06-07,None,None,...,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402781,-83.153634,2017-06-07,None,2017-06-08
6,8418 Fenkell,Able Demolition,2017-08-31,None,2017-05-29,20200.00,None,2017-06-07,None,None,...,012j0000000uc9hAAA,Yes,None,2,Fitzgerald/Marygrove,42.402779,-83.153789,2017-06-07,None,2017-06-09
7,4860 Pennsylvania,Dore & Associates,2017-09-10,2017-06-16,2017-06-16,36000.00,None,2017-07-20,None,None,...,012j0000000uc9hAAA,Yes,None,4,Gratiot Woods,42.381792,-82.997472,2017-07-20,None,None
8,1920 Collingwood,"Adamo Group, Inc.",2017-09-26,None,2017-06-29,114450.00,None,2017-07-13,None,None,...,012j0000000uc9hAAA,Yes,None,5,Dexter-Linwood,42.383735,-83.105180,2017-07-13,None,2017-07-19
9,5942 Cadillac,Rickman Enterprise Group,None,None,2015-10-01,62600.00,2015-10-06,None,None,None,...,012j0000000zM27AAE,Yes,None,4,Gratiot Woods,42.390861,-83.001863,2015-10-06,None,2015-11-19
